# Overview

This notebook demonstrates how to display Earth Engine generated tiles as both static images and on an interactive map.

# Initialize Earth Engine

We start by importing the [Earth Engine Python API](https://pypi.org/project/earthengine-api/) module.

In [ ]:
import ee  # the Earth Engine Python API library

The following command initializes the Earth Engine Python API.

In [ ]:
ee.Initialize()

If the cell produces output that displays an error about needing to authenticate, open up the notebook entitled `01 - Setup auth credentials` and follow the instructions.

# Display a Static Map

This section demonstrates how to obtain a static map image from 
Earth Engine, and display it in a notebook.

The first step is to define an image object. This can be as simple as referencing an image stored in the [Earth Engine Public Data Catalog](https://earthengine.google.com/datasets/) or a computed image that includes several processing steps. The following example uses a reference to the [SRTM Digital Elevation Data 30m](https://explorer.earthengine.google.com/#detail/USGS%2FSRTMGL1_003) dataset.

In [ ]:
# Define an Earth Engine image object.
image = ee.Image("USGS/SRTMGL1_003")

We then can use the [ee.Image.getThumbUrl](https://developers.google.com/earth-engine/api_docs#eeimagegetthumburl) method to construct a URL that will return an image file. 

In [ ]:
url = image.getThumbUrl({'min':0, 'max':3000})
print(url)

Next we use IPython's ability to load and display image files available from a URL (class [IPython.display.Image](https://ipython.org/ipython-doc/3/api/generated/IPython.display.html#IPython.display.Image)).

In [ ]:
from IPython.display import Image
Image(url=url, embed=True, format='png')

# Display a Dynamic Map

To display an Earth Engine generated results on the interactive map, we can use [ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/index.html) and it's [ipyleaflet.TileLayer](https://ipyleaflet.readthedocs.io/en/latest/api_reference/tile_layer.html) class.

In [ ]:
import ipyleaflet  # an interactive mapping "widget" for the notebook

## Displaying a basic interactive map

With ipyleaflet, we can easily create a map and display it in the notebook.

In [ ]:
map1 = ipyleaflet.Map(
    center=(48.2, 16.3), zoom=4,
    layout={'height':'200px'},
)
map1

If you don't like the default background tiles, you can specify a different basemap.

In [ ]:
map2 = ipyleaflet.Map(
    center=(48.2, 16.3), zoom=4,
    layout={'height':'200px'},
    basemap=ipyleaflet.basemaps.Stamen.Toner
)
map2

## Adding Earth Engine generated overlays

First we start by defining a function that takes an Earth Engine image object, and generates a tile layer URL that ipyleaflet can utilize.

In [ ]:
def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)

Next we define a collection of imagery that we want to display. The following creates a collection of imagery from the [Sentinel-2](https://explorer.earthengine.google.com/#detail/COPERNICUS%2FS2) satellite, filtered to a single day.

In [ ]:
s2 = ee.ImageCollection('COPERNICUS/S2').filterDate('2018-01-01', '2018-01-02')

We then can create a tile layer that displays the Sentinel-2 data, and add it to the map.

In [ ]:
sentinel2_tilelayer = ipyleaflet.TileLayer(
    name='Sentinel 2',
    url=GetTileLayerUrl(
        s2.median().visualize(
            min=0,
            max=3000,
            gamma=1.5,
            bands= ['B4', 'B3', 'B2']
        )
    ),
    attribution='Map tiles by <a href="http://earthengine.google.com/">Earth Engine</a>.'
)
map2.add_layer(sentinel2_tilelayer)

We can also add a layer control to our map, the will allow us to toggle the visibility of layers.

In [ ]:
# Adding the layers control to the map.
map2.add_control(ipyleaflet.LayersControl())

The preceding code block did add a layer to a map, but it is a little cumbersome to see because the map was displayed earlier in the notebook and may now be off the screen. If we want, we can display the map in the notebook a second time...

In [ ]:
map2

However, this second map will eventually scroll off the screen as well. Fortunately, there a few ways to keep the output in view when using JupyterLab.

The first method is to open a new content tab that just displays just the cell output. To do this, right-click (or ctrl-click) on the map output (or the code cell that displays the output). In the context menu that appears, select the *"Create New View for Output"* menu item. This will open a new JupyterLab tab below the current tab. Like all JupyterLab tabs, you can resize and rearrange the tabs as needed. The downside of this method is that you need to manually open the new view.

# Using the JupyterLab Sidecar output

The [JupyterLab Sidecar](https://github.com/jupyter-widgets/jupyterlab-sidecar) extension allows you to automatically place output in a collapsible panel on the side of the screen.

In [ ]:
from sidecar import Sidecar

sc = Sidecar(title='Maps!')
with sc:
    print('This content will stay put as your notebook scrolls!')
    display(map1)
    display(map2)

# Linking Map Events

Often it is useful to synchronise the behavior of multiple maps. Because ipywidget maps are a specialized type of Jupyter widget, synchronization can be accomplished using the general [linking widgets](http://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html#Linking-Widgets) approach.



In [ ]:
import ipywidgets
map_zoom_link = ipywidgets.link((map1, 'zoom'), (map2, 'zoom'))
map_center_link = ipywidgets.link((map1, 'center'), (map2, 'center'))

# Split Map Example

The following example shows how to create a map with a splitter control.

In [ ]:
map3 = ipyleaflet.Map(
    center=(48.2, 16.3), zoom=4,
    layout={'height':'500px'},
)

left = ipyleaflet.TileLayer(url='https://cartodb-basemaps-{s}.global.ssl.fastly.net/light_all/{z}/{x}/{y}.png')
control = ipyleaflet.SplitMapControl(left_layer=left, right_layer=sentinel2_tilelayer)
map3.add_control(control)
map3